Working through this notebook on YouTube: https://www.youtube.com/watch?v=c-g6epk3fFE

What is going on everyone and welcome to a video going over the ChatGPT API that was recently released by OpenAI. 

There has been a ChatGPT implementation where you can chat with ChatGPT extremely easily, so why might we be interested in an API instead?

Essentially, the API just plain gives you far more power and control to do more new and novel things with ChatGPT's responses, as well as the ability to integrate it with other applications.

In order to query this model, we will first need an API key. For this, you'll need an account and to set up billing. Typically, you will get some starting credit, but you may or may not, depending on when you sign up and try to use this API. You can create your account at https://platform.openai.com/

From there, go to the top right, click your profile, manage account, and then billing to add a payment method. From here, on the left side, choose API Keys under "user."

Create a key, and then copy the key's value, you will need this in your program. In the same directory that you're working in, create a "key.txt" file and copy and paste the key in there. Save and exit. This particular API costs $0.002, or a fifth of a penny, per 1,000 tokens at the time of my writing.

You will also need the `openai` Python package. You can install it with `pip install --upgrade openai`. The upgrade is there to ensure that you have the latest version, since the ChatGPT API is a new feature.

In [20]:
import openai

# load and set our key
openai.api_key = open("key.txt", "r").read().strip("\n")

The way the ChatGPT API works is you need to query the model. Since these models often make use of chat history/context, every query needs to, or can, include a full message history context. 

Keep in mind, however that the maximum context length is 4096 tokens, so you need to stay under that. There are lots of options to work around this, the simplest being truncating earlier messages, but you can actually even use ChatGPT to help you to summarize and condense the previous message history. Maybe more on this later though. 4096 tokens is something like 20,000 characters, but it this can vary. Tokens are just words, bits of words, or combinations of words or cominations of bits of words. Every response from ChatGPT will inform you how many tokens you're using, so you can keep track.

Let's start with an example input from a user to the API:

In [21]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", # this is "ChatGPT" $0.002 per 1k tokens
  messages=[{"role": "user", "content": "What is the circumference in km of the planet Earth?"}]
)

Notice the "role" is "user." There are 3 roles:

User - This is meant to mimic the end-user that is interacting with the assistant. This is the role that you will be using most of the time.
System - This role can mimic sort of background nudges and prompts that you might want to inject into the conversation, but that dont need a response. At the moment, system is weighted less than "user," so it still seems more useful to use the user for encouraging specific behaviors in my opinion.
Assistant - This is the agent's response. Often this will be actual responses, but keep in mind... you will be able to inject your own responses here, so you can actually have the agent say whatever you want. This is a bit of a hack, but it's a fun one and can be useful in certain situations.

The full completion has a lot of information besides just the text response:

In [22]:
print(completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\nThe circumference of the planet Earth in km is approximately 40,075 km.",
        "role": "assistant"
      }
    }
  ],
  "created": 1678044086,
  "id": "chatcmpl-6qoD8O1qGxluR2fct8hM9aSYDnqzU",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 18,
    "total_tokens": 36
  }
}


In probably most cases, what you're after is specifically:

In [23]:
reply_content = completion.choices[0].message.content
print(reply_content)



The circumference of the planet Earth in km is approximately 40,075 km.


So far so good, this is a very basic example of using the API. In most cases, you're going to need to manage the history, however. The API itself isn't going to manage your history for you, so how might we do that? I would just start with some sort of message history variable for now to keep it simple, but you might use a database or some other storage method. 

In [24]:
message_history = []
# What is the moon's circumference in km?
user_input = input("> ")
print("User's input was: ", user_input)

User's input was:  What is the moon's circumference in km?


Now that you have the user input, let's format it for the API:

In [25]:
message_history.append({"role": "user", "content": f"{user_input}"})

Then we can query the API:

In [26]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_history
)

# Now we can print the response:
reply_content = completion.choices[0].message.content
print(reply_content)



The moon's circumference is approximately 10,917 km.


After getting a repsonse, you'll want to append it to the history:

In [27]:
# note the use of the "assistant" role here. This is because we're feeding the model's response into context.
message_history.append({"role": "assistant", "content": f"{reply_content}"})

We can then followup with another query, demonstrating the use of history:

In [28]:
# which moon is that in reference to?
user_input = input("> ")
print("User's input was: ", user_input)
print()
message_history.append({"role": "user", "content": f"{user_input}"})

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=message_history
)

reply_content = completion.choices[0].message.content
print(reply_content)

User's input was:  which moon is that in reference to?

I'm sorry for the confusion. That circumference is for Earth's moon, also known as Luna.


Combining everything:

In [31]:
message_history = []

def chat(inp, role="user"):
    message_history.append({"role": role, "content": f"{inp}"})
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=message_history
    )
    reply_content = completion.choices[0].message.content
    message_history.append({"role": "assistant", "content": f"{reply_content}"})
    return reply_content

for i in range(2):
    user_input = input("> ")
    print("User's input was: ", user_input)
    print(chat(user_input))
    print()

User's input was:  Can I drink the water from a dehumidifier?


As an AI language model, I cannot recommend that you drink water from a dehumidifier. Although dehumidifiers remove moisture from the air, the water collected can contain impurities and toxins that are harmful to human health. Also, dehumidifiers are not designed to produce potable water, and the water may contain bacteria, fungi, or other contaminants that might make you ill. It is always best to use a certified water source or filter the collected water before drinking it.

User's input was:  How might we make it safe in an emergency to drink?
In an emergency situation, it is essential to have access to safe drinking water. If you need water and there is no other option but to use water from a dehumidifier, here are some steps you can take to make it safer to drink:

1. Check the water: Make sure the water is clean and clear. If the water is cloudy or has particles in it, do not drink it.

2. Boil the water: Boiling the 

Great, looks like everything is working, now, let's see how we might combine this into our own application. We can start off with the most obvious example: A chatbot, and we can make use of `gradio` for the front-end UI.

To use gradio, we'll need to install it with `pip install gradio`.  Then, we'll make our initial imports:

In [ ]:
import gradio as gr
import openai

openai.api_key = open("key.txt", "r").read().strip("\n")

Then, we can start by defining our message history. In this case, let's make our chatbot a joke bot, where we supply the subject(s) and the bot will make a joke from there.

I'll start by having the user submit the following:

"You are a joke bot. I will specify the subject matter in my messages, and you will reply with a joke that includes the subjects I mention in my messages. Reply only with jokes to further input. If you understand, say OK."

In [1]:
message_history = [{"role": "user", "content": f"You are a joke bot. I will specify the subject matter in my messages, and you will reply with a joke that includes the subjects I mention in my messages. Reply only with jokes to further input. If you understand, say OK."},
                   {"role": "assistant", "content": f"OK"}]



Then, we'll inject the assistant's reply of "OK" to encourage it to do what I've asked. Next, we'll make a predict function, which is similar to our `chat` function from before, but is merged with the demo `predict` function from a gradio example:

In [2]:
def predict(input):
    # tokenize the new input sentence
    message_history.append({"role": "user", "content": f"{input}"})

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=message_history
    )
    #Just the reply text
    reply_content = completion.choices[0].message.content#.replace('```python', '<pre>').replace('```', '</pre>')
    
    message_history.append({"role": "assistant", "content": f"{reply_content}"}) 
    
    # get pairs of msg["content"] from message history, skipping the pre-prompt:              here.
    response = [(message_history[i]["content"], message_history[i+1]["content"]) for i in range(2, len(message_history)-1, 2)]  # convert to tuples of list
    return response

Then we can build the gradio app. To make things easier, I'll comment what each line does here:

In [ ]:
# creates a new Blocks app and assigns it to the variable demo.
with gr.Blocks() as demo: 

    # creates a new Chatbot instance and assigns it to the variable chatbot.
    chatbot = gr.Chatbot() 

    # creates a new Row component, which is a container for other components.
    with gr.Row(): 
        '''creates a new Textbox component, which is used to collect user input. 
        The show_label parameter is set to False to hide the label, 
        and the placeholder parameter is set'''
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)
    '''
    sets the submit action of the Textbox to the predict function, 
    which takes the input from the Textbox, the chatbot instance, 
    and the state instance as arguments. 
    This function processes the input and generates a response from the chatbot, 
    which is displayed in the output area.'''
    txt.submit(predict, txt, chatbot) # submit(function, input, output)
    #txt.submit(lambda :"", None, txt)  #Sets submit action to lambda function that returns empty string 

    '''
    sets the submit action of the Textbox to a JavaScript function that returns an empty string. 
    This line is equivalent to the commented out line above, but uses a different implementation. 
    The _js parameter is used to pass a JavaScript function to the submit method.'''
    txt.submit(None, None, txt, _js="() => {''}") # No function, no input to that function, submit action to textbox is a js function that returns empty string, so it clears immediately.
         
demo.launch()

The full app now is:

In [ ]:
import gradio as gr
import openai

openai.api_key = open("key.txt", "r").read().strip("\n")

message_history = [{"role": "user", "content": f"You are a joke bot. I will specify the subject matter in my messages, and you will reply with a joke that includes the subjects I mention in my messages. Reply only with jokes to further input. If you understand, say OK."},
                   {"role": "assistant", "content": f"OK"}]

def predict(input):
    # tokenize the new input sentence
    message_history.append({"role": "user", "content": f"{input}"})

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo", #10x cheaper than davinci, and better. $0.002 per 1k tokens
      messages=message_history
    )
    #Just the reply:
    reply_content = completion.choices[0].message.content#.replace('```python', '<pre>').replace('```', '</pre>')

    print(reply_content)
    message_history.append({"role": "assistant", "content": f"{reply_content}"}) 
    
    # get pairs of msg["content"] from message history, skipping the pre-prompt:              here.
    response = [(message_history[i]["content"], message_history[i+1]["content"]) for i in range(2, len(message_history)-1, 2)]  # convert to tuples of list
    return response

# creates a new Blocks app and assigns it to the variable demo.
with gr.Blocks() as demo: 

    # creates a new Chatbot instance and assigns it to the variable chatbot.
    chatbot = gr.Chatbot() 

    # creates a new Row component, which is a container for other components.
    with gr.Row(): 
        '''creates a new Textbox component, which is used to collect user input. 
        The show_label parameter is set to False to hide the label, 
        and the placeholder parameter is set'''
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)
    '''
    sets the submit action of the Textbox to the predict function, 
    which takes the input from the Textbox, the chatbot instance, 
    and the state instance as arguments. 
    This function processes the input and generates a response from the chatbot, 
    which is displayed in the output area.'''
    txt.submit(predict, txt, chatbot) # submit(function, input, output)
    #txt.submit(lambda :"", None, txt)  #Sets submit action to lambda function that returns empty string 

    '''
    sets the submit action of the Textbox to a JavaScript function that returns an empty string. 
    This line is equivalent to the commented out line above, but uses a different implementation. 
    The _js parameter is used to pass a JavaScript function to the submit method.'''
    txt.submit(None, None, txt, _js="() => {''}") # No function, no input to that function, submit action to textbox is a js function that returns empty string, so it clears immediately.
         
demo.launch()

From here, we can open the app:

```
$ python3 gradio-joke.py 
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
```

Now, you could input something like: 

`Programmers and boats`

The response I got with this was:

`Why did the programmer quit his job on the boat? He found the C to shining C.`

You will get something different most likely, but you can try anything you want, it could be a single subject, or even 3 or more different subjects. For example, a single subject:

`Lego` > `Why don't Lego characters have girlfriends? Because they block all the relationships.`

Or many subjects:

`Python, Java, and C++` > `Why did Python break up with Java and C++? Because they were too strongly typed for Python's taste!`

Not all jokes are "good" and sometimes ChatGPT seems to just make 2 jokes. You could probably further pre-promopt to stop that behavior, but you get the idea. This is just one example of creating a very basic application with the ChatGPT API. There's a whole lot more interesting things that we can do, and I have a few more specific and in depth ideas for projects that I'll be working on 


